In [ ]:
from utils import *

In [3]:
df = load_filtered_data(years=('2019',), folder='../AIS_data', min_data_points=100)

In [4]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
from sklearn.utils import resample



class TripDataset(Dataset):
    def __init__(self, df):
        self.data = []
        self.labels = []
        for _, row in df.iterrows():
            seq = torch.tensor(list(zip(row['elapsed_s'], row['LAT'], row['LON'])), dtype=torch.float32)
            self.data.append(seq)
            self.labels.append(int(row['Label']))
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

def collate_fn(batch):
    sequences, labels = zip(*batch)
    padded = pad_sequence(sequences, batch_first=True)
    lengths = torch.tensor([len(seq) for seq in sequences])
    mask = torch.arange(padded.size(1))[None, :] < lengths[:, None]
    return padded, torch.tensor(labels), mask

class TCNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size= 10, dilation=1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size,
                      padding=(kernel_size - 1) * dilation, dilation=dilation),
            nn.ReLU(),
            nn.Conv1d(out_channels, out_channels, kernel_size,
                      padding=(kernel_size - 1) * dilation, dilation=dilation),
            nn.ReLU()
        )

    def forward(self, x):
        return self.net(x)

class TCNClassifier(nn.Module):
    def __init__(self, input_dim=3, hidden_dim=32, num_blocks=3):
        super().__init__()
        layers = []
        for i in range(num_blocks):
            dilation = 2 ** i
            in_dim = input_dim if i == 0 else hidden_dim
            layers.append(TCNBlock(in_dim, hidden_dim, dilation=dilation))
        self.tcn = nn.Sequential(*layers)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, 1),
        )

    def forward(self, x, mask=None):
        # x: (batch, seq_len, input_dim) -> (batch, input_dim, seq_len)
        x = x.transpose(1, 2)
        out = self.tcn(x)
        out = out.mean(dim=2)  # Global average pooling over time
        return self.classifier(out).squeeze(-1)


# Prepare data loaders
# Stratified split on the DataFrame before dataset creation
train_df, val_df = train_test_split(
    df,
    stratify=df['Label'],
    test_size=0.2,
    random_state=42
)

# Separate majority and minority classes
majority_class = train_df[train_df['Label'] == 0]
minority_class = train_df[train_df['Label'] == 1]

# Downsample majority class
majority_downsampled = resample(
    majority_class,
    replace=False,                # sample without replacement
    n_samples=len(minority_class),  # match minority class count
    random_state=42
)

# Combine minority class with downsampled majority class
train_df_balanced = pd.concat([majority_downsampled, minority_class])



# Create datasets
train_ds = TripDataset(train_df_balanced)
val_ds   = TripDataset(val_df)

# Create dataloaders
train_loader = DataLoader(train_ds, batch_size=1024, shuffle=True, collate_fn=collate_fn)
val_loader   = DataLoader(val_ds, batch_size=1024, shuffle=False, collate_fn=collate_fn)



In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TCNClassifier().to(device)

# Weighted BCE loss
labels = [label for _, label in train_ds]
counts = np.bincount(labels)
pos_weight = torch.tensor(counts[0] / (counts[1] + 1e-6)).float().to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

best_f1 = 0
for epoch in range(21):
    model.train()
    for x, y, mask in train_loader:
        x, y, mask = x.to(device), y.float().to(device), mask.to(device)
        logits = model(x, mask)
        loss = criterion(logits, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for x, y, mask in val_loader:
            x, y, mask = x.to(device), y.float().to(device), mask.to(device)
            preds = (torch.sigmoid(model(x, mask)) > 0.5).long()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())
    epoch_f1 = f1_score(all_labels, all_preds)
    print(f"Epoch {epoch+1} Validation F1: {epoch_f1:.4f}")
    if epoch_f1 > best_f1:
        best_f1 = epoch_f1
        torch.save(model.state_dict(), "best_model.pt")

print("Best Validation F1:", best_f1)


Epoch 1 Validation F1: 0.0349
Epoch 2 Validation F1: 0.0114
Epoch 3 Validation F1: 0.0178
Epoch 4 Validation F1: 0.0226
Epoch 5 Validation F1: 0.0114
Epoch 6 Validation F1: 0.0248
Epoch 7 Validation F1: 0.0126
Epoch 8 Validation F1: 0.0237
Epoch 9 Validation F1: 0.0185
Epoch 10 Validation F1: 0.0185
Epoch 11 Validation F1: 0.0162
Epoch 12 Validation F1: 0.0194
Epoch 13 Validation F1: 0.0193
Epoch 14 Validation F1: 0.0192
Epoch 15 Validation F1: 0.0192
Epoch 16 Validation F1: 0.0192
Epoch 17 Validation F1: 0.0191
Epoch 18 Validation F1: 0.0190
Epoch 19 Validation F1: 0.0188
Epoch 20 Validation F1: 0.0188
Epoch 21 Validation F1: 0.0190
Best Validation F1: 0.03493191237418591
